# Basic Tutorial for Polygenic Risk Score Analysis

https://choishingwan.github.io/PRS-Tutorial/

今回はこれにしたがって動かしてみます

## 1. QC of Base Data

### Obtaining the base data file

In [4]:
#データのダウンロード
cd ~/hmiwa/hmiwa-lab/d_20231023/03_231113_TutorialPRS
mkdir -p data
cd data
wget "https://drive.google.com/u/0/uc?id=1RWjk49QNZj9zvJHc9X_wyZ51fdy6xQjv&export=download" -O Height.gwas.txt.gz
ls

--2023-11-13 01:50:18--  https://drive.google.com/u/0/uc?id=1RWjk49QNZj9zvJHc9X_wyZ51fdy6xQjv&export=download
Resolving drive.google.com (drive.google.com)... 172.217.26.238, 2404:6800:4004:801::200e
Connecting to drive.google.com (drive.google.com)|172.217.26.238|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.google.com/uc?id=1RWjk49QNZj9zvJHc9X_wyZ51fdy6xQjv&export=download [following]
--2023-11-13 01:50:18--  https://drive.google.com/uc?id=1RWjk49QNZj9zvJHc9X_wyZ51fdy6xQjv&export=download
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-bo-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i0q63k8tmcl049rvs9m96mh6os68hsoe/1699807800000/00063656433649606985/*/1RWjk49QNZj9zvJHc9X_wyZ51fdy6xQjv?e=download&uuid=e2ae40ef-b410-41e3-ae97-aa5399397f4c [following]
--2023-11-13 01:50:19--  https://doc-08-bo-docs.googleusercontent.com/docs/secu

注）google ドライブからwgetで取得するヒントはこちら

https://qiita.com/namakemono/items/c963e75e0af3f7eed732

### Reading the base data file

In [5]:
#なかみの確認
gunzip -c Height.gwas.txt.gz | head

CHR	BP	SNP	A1	A2	N	SE	P	OR	INFO	MAF
1	756604	rs3131962	A	G	388028	0.00301666	0.483171	0.997886915712657	0.890557941364774	0.369389592764921
1	768448	rs12562034	A	G	388028	0.00329472	0.834808	1.00068731609353	0.895893511351165	0.336845754096289
1	779322	rs4040617	G	A	388028	0.00303344	0.42897	0.997603556067569	0.897508290615237	0.377368010940814
1	801536	rs79373928	G	T	388028	0.00841324	0.808999	1.00203569922793	0.908962856432993	0.483212245374095
1	808631	rs11240779	G	A	388028	0.00242821	0.590265	1.00130832511154	0.893212523690488	0.450409558999587
1	809876	rs57181708	G	A	388028	0.00336785	0.71475	1.00123165786833	0.923557624081969	0.499743932656759
1	835499	rs4422948	G	A	388028	0.0023758	0.710884	0.999119752645202	0.906437735120596	0.481016005816168
1	838555	rs4970383	A	C	388028	0.00235773	0.150993	0.996619945289758	0.907716506801574	0.327164029672754
1	840753	rs4970382	C	T	388028	0.00207377	0.199967	0.99734567895614	0.914602590137255	0.498936220426316

gzip: stdout: Broken pipe


列の内容について：（サイトから引用）
* CHR: The chromosome in which the SNP resides
* BP: Chromosomal co-ordinate of the SNP
* SNP: SNP ID, usually in the form of rs-ID
* A1: The effect allele of the SNP
* A2: The non-effect allele of the SNP
* N: Number of samples used to obtain the effect size estimate
* SE: The standard error (SE) of the effect size esimate
* P: The P-value of association between the SNP genotypes and the base phenotype
* OR: The effect size estimate of the SNP, if the outcome is binary/case-control. If the outcome is continuous or treated as continuous then this will usually be BETA
* INFO: The imputation information score
* MAF: The minor allele frequency (MAF) of the SNP

### QC checklist: Base data

#### Heritability check

We recommend that PRS analyses are performed on base data with a chip-heritability estimate h2snp>0.05. The chip-heritability of a GWAS can be estimated using e.g. LD Score Regression (LDSC). Our height GWAS data are simulated to have a chip-heritability much greater than 0.05 and so we can move on to the next QC step.

PRS解析はチップヘリタビリティ推定値h2snp>0.05のベースデータで行うことを推奨する。GWASのチップヘリタビリティーは、例えばLD Score Regression (LDSC)を用いて推定することができる。我々の身長GWASデータは、0.05よりはるかに大きいチップヘリタビリティーを持つようにシミュレートされているので、次のQCステップに進むことができる。

#### Effect allele

It is important to know which allele is the effect allele and which is the non-effect allele for PRS association results to be in the correct direction.

PRSの関連結果を正しい方向に導くためには、どの対立遺伝子が効果対立遺伝子で、どの対立遺伝子が非効果対立遺伝子かを知ることが重要である。

>Some GWAS results files do not make clear which allele is the effect allele and which is the non-effect allele. If the incorrect assumption is made in computing the PRS, then the effect of the PRS in the target data will be in the wrong direction.
>
>GWASの結果ファイルの中には、どの対立遺伝子が効果対立遺伝子で、どの対立遺伝子が非効果対立遺伝子なのかを明確にしていないものがある。PRSを計算する際に間違った仮定がなされると、対象データにおけるPRSの効果は間違った方向になる。
>
>To avoid misleading conclusions the effect allele from the base (GWAS) data must be known.
>
>誤解を招く結論を避けるためには、ベース（GWAS）データからの効果対立遺伝子を知っていなければならない。

#### File transfer

A common problem is that the downloaded base data file can be corrupted during download, which can cause PRS software to crash or to produce errors in results. However, a md5sum hash is generally included in files so that file integrity can be checked. The following command performs this md5sum check:

よくある問題は、ダウンロードしたベースデータファイルがダウンロード中に破損し、PRSソフトウェアがクラッシュしたり、結果にエラーが出ることです。しかし、ファイルの完全性をチェックできるように、一般的にファイルにはmd5sumハッシュが含まれています。次のコマンドは、このmd5sumチェックを実行します：

In [7]:
md5sum Height.gwas.txt.gz

a2b15fb6a2bbbe7ef49f67959b43b160  Height.gwas.txt.gz


if the file is intact, then md5sum generates a string of characters, which in this case should be: a2b15fb6a2bbbe7ef49f67959b43b160. If a different string is generated, then the file is corrupted.

ファイルが無傷の場合、md5sumは文字列を生成し、この場合a2b15fb6a2bbbe7ef49f67959b43b160となる。異なる文字列が生成された場合、ファイルは壊れている。

#### Genome build

The height summary statistic are on the same genome build as the target data that we will be using. You must check that your base and target data are on the same genome build, and if they are not then use a tool such as LiftOver to make the builds consistent across the data sets.

高さの要約統計量は、これから使用するターゲットデータと同じゲノムビルドにあります。ベースデータとターゲットデータが同じゲノムビルド上にあることを確認し、もしそうでない場合は、LiftOverのようなツールを使ってデータセット間でビルドを一貫させる必要があります。

LiftOver:https://genome.ucsc.edu/cgi-bin/hgLiftOver

#### Standard GWAS QC

As described in the paper, both the base and target data should be subjected to the standard stringent QC steps performed in GWAS. If the base data have been obtained as summary statistics from a public source, then the typical QC steps that you will be able to perform on them are to filter the SNPs according to INFO score and MAF. SNPs with low minor allele frequency (MAF) or imputation information score (INFO) are more likely to generate false positive results due to their lower statistical power (and higher probability of genotyping errors in the case of low MAF). Therefore, SNPs with low MAF and INFO are typically removed before performing downstream analyses. We recommend removing SNPs with MAF < 1% and INFO < 0.8 (with very large base sample sizes these thresholds could be reduced if sensitivity checks indicate reliable results). These SNP filters can be achieved using the following code:

論文に記載されているように、ベースデータとターゲットデータの両方は、GWASで実行される標準的な厳格なQCステップに従うべきである。もしベースデータが公開されているソースから要約統計として取得されたものであれば、それに対して実行できる典型的なQCステップは、INFOスコアとMAFに従ってSNPをフィルタリングすることである。マイナーアレル頻度（MAF）またはインピュテーション情報スコア（INFO）が低いSNPは、統計的検出力が低い（MAFが低い場合はジェノタイピングエラーの確率が高い）ため、偽陽性の結果を生成する可能性が高くなります。したがって、MAFやINFOが低いSNPは、ダウンストリーム解析を行う前に除去するのが一般的である。MAF＜1％、INFO＜0.8のSNPを除去することを推奨する（ベースサンプルサイズが非常に大きい場合、感度のチェックで信頼できる結果が示されれば、これらの閾値を下げることができる）。これらのSNPフィルターは以下のコードで実現できます：

In [8]:
gunzip -c Height.gwas.txt.gz |\
awk 'NR==1 || ($11 > 0.01) && ($10 > 0.8) {print}' |\
gzip  > Height.gz

In [9]:
ls

Height.gwas.txt.gz  Height.gz


The bash code above does the following:

1. Decompresses and reads the Height.gwas.txt.gz file
2. Prints the header line (NR==1)
3. Prints any line with MAF above 0.01 (\$11 because the eleventh column of the file contains the MAF information)
4. Prints any line with INFO above 0.8 (\$10 because the tenth column of the file contains the INFO information)
5. Compresses and writes the results to Height.gz

#### Mismatching SNPs

SNPs that have mismatching alleles reported in the base and target data are either resolvable by "strand-flipping" the alleles to their complementary alleles in e.g. the target data, such as for a SNP with A/C in the base data and G/T in the target, or non-resolvable, such as for a SNP with C/G in the base and C/T in the target. Most polygenic score software perform strand-flipping automatically for SNPs that are resolvable, and remove non-resolvable mismatching SNPs.

ベースデータとターゲットデータで報告された対立遺伝子が不一致のSNPは、例えばベースデータでA/C、ターゲットデータでG/TのSNPのように、ターゲットデータで対立遺伝子を相補的な対立遺伝子に "ストランドフリップ "することによって解決可能か、あるいはベースデータでC/G、ターゲットデータでC/TのSNPのように解決不可能かのいずれかである。ほとんどの多遺伝子スコアソフトは、解決可能なSNPに対しては自動的にストランドフリッピングを行い、解決不可能なミスマッチSNPは削除する。

Since we need the target data to know which SNPs have mismatching alleles, we will perform this strand-flipping in the target data.

どのSNPがミスマッチした対立遺伝子を持つかを知るためにはターゲットデータが必要なので、このストランドフリッピングをターゲットデータで行う。

#### Duplicate SNPs

If an error has occurred in the generation of the base data then there may be duplicated SNPs in the base data file. Most PRS software do not allow duplicated SNPs in the base data input and thus they should be removed, using a command such as the one below:

ベースデータ作成時にエラーが発生した場合、ベースデータファイル中に重複した SNP が存在する可能性がある。ほとんどのPRSソフトウェアでは、重複SNPのベースデータ入力を許可していないため、以下のようなコマンドを使用して、重複SNPを削除する必要があります：

In [10]:
gunzip -c Height.gz |\
awk '{seen[$3]++; if(seen[$3]==1){ print}}' |\
gzip - > Height.nodup.gz

In [11]:
ls

Height.gwas.txt.gz  Height.gz  Height.nodup.gz


The above command does the following:

1. Decompresses and reads the Height.gz file
2. Count number of time SNP ID was observed, assuming the third column contian the SNP ID (seen[$3]++). If this is the first time seeing this SNP ID, print it.
3. Compresses and writes the results to Height.nodup.gz


In [18]:
echo "Height.gz"
gunzip -c Height.gz | head -n 5
echo "count:"`gunzip -c Height.gz | wc | awk '{print $1}'`
echo "Height.nodup.gz"
gunzip -c Height.nodup.gz | head -n 5
echo "count:"`gunzip -c Height.nodup.gz | wc | awk '{print $1}'`

Height.gz
CHR	BP	SNP	A1	A2	N	SE	P	OR	INFO	MAF
1	756604	rs3131962	A	G	388028	0.00301666	0.483171	0.997886915712657	0.890557941364774	0.369389592764921
1	768448	rs12562034	A	G	388028	0.00329472	0.834808	1.00068731609353	0.895893511351165	0.336845754096289
1	779322	rs4040617	G	A	388028	0.00303344	0.42897	0.997603556067569	0.897508290615237	0.377368010940814
1	801536	rs79373928	G	T	388028	0.00841324	0.808999	1.00203569922793	0.908962856432993	0.483212245374095

gzip: stdout: Broken pipe
count:529496
Height.nodup.gz
CHR	BP	SNP	A1	A2	N	SE	P	OR	INFO	MAF
1	756604	rs3131962	A	G	388028	0.00301666	0.483171	0.997886915712657	0.890557941364774	0.369389592764921
1	768448	rs12562034	A	G	388028	0.00329472	0.834808	1.00068731609353	0.895893511351165	0.336845754096289
1	779322	rs4040617	G	A	388028	0.00303344	0.42897	0.997603556067569	0.897508290615237	0.377368010940814
1	801536	rs79373928	G	T	388028	0.00841324	0.808999	1.00203569922793	0.908962856432993	0.483212245374095

gzip: stdout: Broken pipe
count

#### Ambiguous SNPs

If the base and target data were generated using different genotyping chips and the chromosome strand (+/-) that was used for either is unknown, then it is not possible to pair-up the alleles of ambiguous SNPs (i.e. those with complementary alleles, either C/G or A/T SNPs) across the data sets, because it will be unknown whether the base and target data are referring to the same allele or not. While allele frequencies could be used to infer which alleles are on the same strand, the accuracy of this could be low for SNPs with MAF close to 50% or when the base and target data are from different populations. Therefore, we recommend removing all ambiguous SNPs to avoid introducing this potential source of systematic error.

ベースとターゲットのデータが異なるジェノタイピングチップを用いて作成され、どちらかに使用された染色体鎖（+/-）が不明である場合、ベースとターゲットのデータが同じ対立遺伝子を参照しているかどうかが不明であるため、データセット間であいまいなSNP（すなわち、相補的な対立遺伝子を持つSNP、C/GまたはA/TのSNP）の対立遺伝子をペアリングすることはできない。どの対立遺伝子が同じ鎖上にあるのかを推測するために対立遺伝子頻度を用いることもできるが、MAFが50%に近いSNPや、ベースデータとターゲットデータが異なる集団のものである場合、その精度は低くなる可能性がある。したがって、このような系統誤差の原因となる可能性を避けるために、あいまいなSNPはすべて削除することを推奨する。

Ambiguous SNPs can be removed in the base data and then there will be no such SNPs in the subsequent analyses, since analyses are performed only on SNPs that overlap between the base and target data. Nonambiguous SNPs can be retained using the following:

曖昧なSNPはベースデータから削除することができ、ベースデータとターゲットデータの間で重複するSNPに対してのみ解析が行われるため、その後の解析ではそのようなSNPは存在しない。曖昧でないSNPは以下の方法で残すことができる：

In [19]:
gunzip -c Height.nodup.gz |\
awk '!( ($4=="A" && $5=="T") || \
        ($4=="T" && $5=="A") || \
        ($4=="G" && $5=="C") || \
        ($4=="C" && $5=="G")) {print}' |\
    gzip > Height.QC.gz

In [20]:
gunzip -c Height.QC.gz | wc | awk '{print $1}'

499618


#### Sex chromosomes

Sometimes sample mislabelling can occur, which may lead to invalid results. One indication of a mislabelled sample is a difference between reported sex and that indicated by the sex chromosomes. While this may be due to a difference in sex and gender identity, it could also reflect mislabeling of samples or misreporting and, thus, individuals in which there is a mismatch between biological and reported sex are typically removed. See the Target Data section in which a sex-check is performed.

時には検体の誤標識が起こり、無効な結果につながることがあります。誤ラベリングされたサンプルの兆候の一つは、報告された性と性染色体が示す性との違いである。これは性別や性自認の違いによるものかもしれないが、サンプルの誤ラベリングや報告ミスを反映している可能性もあるため、生物学的性別と報告された性別が不一致の個体は通常除外される。性別チェックが行われる「対象データ」の項を参照のこと。

#### Sample overlap

Since the target data were simulated there are no overlapping samples between the base and target data here (see the relevant section of the paper for discussion of the importance of avoiding sample overlap).

ターゲットデータはシミュレートされたものであるため、ベースデータとターゲットデータの間に重複するサンプルはない（サンプルの重複を避けることの重要性については、論文の関連セクションを参照）。

#### Relatedness

Closely related individuals within and between the base and the target data may lead to overfitted results, limiting the generalizability of the results (see the relevant sections of the paper). Relatedness within the target data is tested in the Target Data section.

ベースデータ内およびターゲットデータ間で密接に関連する個体は、結果の一般化可能性を制限する過剰適合につながる可能性がある（論文の関連セクションを参照）。ターゲットデータ内の関連性は、ターゲットデータのセクションでテストされます。

**The Height.QC.gz base data are now ready for using in downstream analyses.**

これで、Height.QC.gzベースデータは、下流の解析で使用する準備が整いました。

## 2. QC of Target Data

### Obtaining the target data

In [29]:
#データのダウンロード
#https://drive.google.com/u/0/uc?id=1uhJR_3sn7RA8U5iYQbcmTp6vFdQiF4F2&export=download
FILE_ID=1uhJR_3sn7RA8U5iYQbcmTp6vFdQiF4F2
FILE_NAME=EUR.zip
curl -sc /tmp/cookie "https://drive.google.com/uc?export=download&id=${FILE_ID}" > /dev/null
CODE="$(awk '/_warning_/ {print $NF}' /tmp/cookie)"  
curl -Lb /tmp/cookie "https://drive.google.com/uc?export=download&confirm=${CODE}&id=${FILE_ID}" -o ${FILE_NAME}
ls

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 36.6M  100 36.6M    0     0  7245k      0  0:00:05  0:00:05 --:--:-- 9962k
EUR.zip  Height.gwas.txt.gz  Height.gz  Height.nodup.gz  Height.QC.gz


In [30]:
file EUR.zip
unzip EUR.zip
ls

EUR.zip: Zip archive data, at least v2.0 to extract, compression method=deflate
Archive:  EUR.zip
  inflating: EUR.bed                 
  inflating: EUR.bim                 
  inflating: EUR.cov                 
  inflating: EUR.fam                 
  inflating: EUR.height              
EUR.bed  EUR.cov  EUR.height  Height.gwas.txt.gz  Height.nodup.gz
EUR.bim  EUR.fam  EUR.zip     Height.gz           Height.QC.gz


### QC checklist: Target data
#### Sample size
We recommend that users only perform PRS analyses on target data of at least 100 individuals. The sample size of our target data here is 503 individuals.

少なくとも100人以上の対象データに対してのみPRS分析を行うことを推奨する。ここでの対象データのサンプルサイズは503人である。
#### File transfer
Usually we do not need to download and transfer the target data file because it is typically generated locally. However, the file should contain an md5sum code in case we send the data file to collaborators who may want to confirm that the file has not changed during the transfer.

通常、対象のデータファイルはローカルで生成されるため、ダウンロードして転送する必要はない。しかし、転送中にファイルが変更されていないことを確認したい共同研究者にデータファイルを送る場合に備えて、ファイルにはmd5sumコードを含める必要がある。

In [31]:
md5sum EUR*

98bcef133f683b1272d3ea5f97742e0e  EUR.bed
6b286002904880055a9c94e01522f059  EUR.bim
85ed18288c708e095385418517e9c3bd  EUR.cov
e7b856f0c7bcaffc8405926d08386e97  EUR.fam
dd445ce969a81cded20da5c88b82d4df  EUR.height
833aa8ee3a4d9fb591ed61ba44eebe1b  EUR.zip


#### Genome build
As stated in the base data section, the genome build for our base and target data is the same, as it should be.

ベースデータのセクションで述べたように、ベースデータとターゲットデータのゲノムビルドは同じである。
#### Standard GWAS QC
The target data must be quality controlled to at least the standards implemented in GWAS studies, e.g. removing SNPs with low genotyping rate, low minor allele frequency, out of Hardy-Weinberg Equilibrium, removing individuals with low genotyping rate (see Marees et al).The following plink command applies some of these QC metrics to the target data:

対象データは、少なくともGWAS研究で実施されている基準で品質管理されていなければならない。例えば、ジェノタイピング率が低いSNP、マイナーアレル頻度が低いSNP、ハーディーワインベルグ平衡から外れたSNP、ジェノタイピング率が低い個体の除去などである（Marees et alを参照）。以下のplinkコマンドは、これらのQCメトリクスの一部を対象データに適用する：

In [32]:
plink \
    --bfile EUR \
    --maf 0.01 \
    --hwe 1e-6 \
    --geno 0.01 \
    --mind 0.01 \
    --write-snplist \
    --make-just-fam \
    --out EUR.QC

PLINK v1.90b7 64-bit (16 Jan 2023)             www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to EUR.QC.log.
Options in effect:
  --bfile EUR
  --geno 0.01
  --hwe 1e-6
  --maf 0.01
  --make-just-fam
  --mind 0.01
  --out EUR.QC
  --write-snplist

257564 MB RAM detected; reserving 128782 MB for main workspace.
551892 variants loaded from .bim file.
503 people (240 males, 263 females) loaded from .fam.
14 people removed due to missing genotype data (--mind).
IDs written to EUR.QC.irem .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 489 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
treat these as missing.
Total genotyping rate in remaining samples is 0.999816.
5353 variant

>Normally, we can generate a new genotype file using the new sample list. However, this will use up a lot of storage space. Using plink's --extract, --exclude, --keep, --remove, --make-just-fam and --write-snplist functions, we can work solely on the list of samples and SNPs without duplicating the genotype file, reducing the storage space usage.
>
>通常、新しいサンプルリストを用いて新しい遺伝子型ファイルを生成することができます。しかし、これでは多くの記憶容量を使うことになる。plinkの--extract関数、--exclude関数、--keep関数、--remove関数、--make-just-fam関数、--write-snplist関数を使えば、遺伝子型ファイルを複製することなく、サンプルとSNPのリストだけで作業することができ、ストレージの使用量を減らすことができる。

Very high or low heterozygosity rates in individuals could be due to DNA contamination or to high levels of inbreeding. Therefore, samples with extreme heterozygosity are typically removed prior to downstream analyses.

個体内のヘテロ接合率が非常に高いか低いのは、DNAのコンタミネーションや近親交配が多いためである可能性がある。従って、極端なヘテロ接合率を持つサンプルは、通常、下流の解析の前に除去される。

First, we perform pruning to remove highly correlated SNPs:

まず、相関性の高いSNPを除去するためにプルーニングを行う：

In [33]:
plink \
    --bfile EUR \
    --keep EUR.QC.fam \
    --extract EUR.QC.snplist \
    --indep-pairwise 200 50 0.25 \
    --out EUR.QC

PLINK v1.90b7 64-bit (16 Jan 2023)             www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to EUR.QC.log.
Options in effect:
  --bfile EUR
  --extract EUR.QC.snplist
  --indep-pairwise 200 50 0.25
  --keep EUR.QC.fam
  --out EUR.QC

257564 MB RAM detected; reserving 128782 MB for main workspace.
551892 variants loaded from .bim file.
503 people (240 males, 263 females) loaded from .fam.
--extract: 540534 variants remaining.
--keep: 489 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 489 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
treat these as missing.
Total genotyping rate in remaining samples is 0.999919.
540534 variants and 489 people pass filter

This will generate two files 1) EUR.QC.prune.in and 2) EUR.QC.prune.out. All SNPs within EUR.QC.prune.in have a pairwise r2<0.25.

これにより2つのファイル1) EUR.QC.prune.inと2) EUR.QC.prune.outが生成される。EUR.QC.prune.in内のSNPはすべてペアワイズr2<0.25である。

Heterozygosity rates can then be computed using plink:

ヘテロ接合率はplinkを使って計算できる：

In [34]:
plink \
    --bfile EUR \
    --extract EUR.QC.prune.in \
    --keep EUR.QC.fam \
    --het \
    --out EUR.QC

PLINK v1.90b7 64-bit (16 Jan 2023)             www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to EUR.QC.log.
Options in effect:
  --bfile EUR
  --extract EUR.QC.prune.in
  --het
  --keep EUR.QC.fam
  --out EUR.QC

257564 MB RAM detected; reserving 128782 MB for main workspace.
551892 variants loaded from .bim file.
503 people (240 males, 263 females) loaded from .fam.
--extract: 268457 variants remaining.
--keep: 489 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 489 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
treat these as missing.
Total genotyping rate in remaining samples is 0.999958.
268457 variants and 489 people pass filters and QC.
Note: No phe

This will generate the EUR.QC.het file, which contains F coefficient estimates for assessing heterozygosity. We will remove individuals with F coefficients that are more than 3 standard deviation (SD) units from the mean, which can be performed using the following R command (assuming that you have R downloaded, then you can open an R session by typing R in your terminal):

これはヘテロ接合性を評価するためのF係数推定値を含むEUR.QC.hetファイルを生成する。平均から3標準偏差(SD)単位以上のF係数を持つ個体を削除します。これは、以下のRコマンドで実行できます（Rがダウンロードされていることを仮定して、ターミナルでRとタイプしてRセッションを開くことができます）：

In [36]:
echo 'dat <- read.table("EUR.QC.het", header=T) # Read in the EUR.het file, specify it has header' > het.R
echo 'm <- mean(dat$F) # Calculate the mean' >> het.R
echo 's <- sd(dat$F) # Calculate the SD' >> het.R
echo 'valid <- subset(dat, F <= m+3*s & F >= m-3*s) # Get any samples with F coefficient within 3 SD of the population mean' >> het.R
echo 'write.table(valid[,c(1,2)], "EUR.valid.sample", quote=F, row.names=F) # print FID and IID for valid samples' >> het.R

Rscript --no-save het.R

In [37]:
ls

EUR.bed  EUR.height  EUR.QC.irem       EUR.QC.snplist      Height.gz
EUR.bim  EUR.QC.fam  EUR.QC.log        EUR.valid.sample    Height.nodup.gz
EUR.cov  EUR.QC.het  EUR.QC.prune.in   EUR.zip             Height.QC.gz
EUR.fam  EUR.QC.hh   EUR.QC.prune.out  Height.gwas.txt.gz  het.R


In [38]:
head EUR.valid.sample

FID IID
HG00096 HG00096
HG00097 HG00097
HG00099 HG00099
HG00101 HG00101
HG00102 HG00102
HG00103 HG00103
HG00105 HG00105
HG00107 HG00107
HG00108 HG00108


In [39]:
wc EUR.valid.sample
wc EUR.QC.het

 488  976 7800 EUR.valid.sample
  490  2940 35280 EUR.QC.het


#### Ambiguous SNPs
These were removed during the base data QC.

これらはベースデータのQCで削除された。

**【重要】**

**ここまでのファイルをhmiwa-lab直下でやってしまったが、大きなデータはhmiwa1の方へ移しておかねばならなかった**

In [1]:
cd ~/hmiwa/hmiwa-lab/d_20231023/03_231113_TutorialPRS
mkdir -p ~/hmiwa1/storage/hmiwa-lab/03_231113_TutorialPRS
mv data ~/hmiwa1/storage/hmiwa-lab/03_231113_TutorialPRS/data

In [3]:
echo "hmiwa-lab"
ls ~/hmiwa/hmiwa-lab/d_20231023/03_231113_TutorialPRS
echo "hmiwa1"
ls ~/hmiwa1/storage/hmiwa-lab/03_231113_TutorialPRS/data

hmiwa-lab
PRSTutorial1103.ipynb
hmiwa1
EUR.bed  EUR.height  EUR.QC.irem       EUR.QC.snplist      Height.gz
EUR.bim  EUR.QC.fam  EUR.QC.log        EUR.valid.sample    Height.nodup.gz
EUR.cov  EUR.QC.het  EUR.QC.prune.in   EUR.zip             Height.QC.gz
EUR.fam  EUR.QC.hh   EUR.QC.prune.out  Height.gwas.txt.gz  het.R


#### Mismatching SNPs
SNPs that have mismatching alleles reported in the base and target data may be resolvable by strand-flipping the alleles to their complementary alleles in e.g. the target data, such as for a SNP with A/C in the base data and G/T in the target. This can be achieved with the following steps:

ベースデータとターゲットデータで報告されたアレルが不一致のSNPは、例えば、ベースデータでA/C、ターゲットデータでG/TのSNPのように、アレルがターゲットデータで相補的なアレルにストランドフリップすることで解決できる場合がある。これは以下のステップで達成できる：

>Most PRS software will perform strand-flipping automatically, thus this step is usually not required.
>
>ほとんどのPRSソフトウェアは自動的にストランドフリップを行うので、通常このステップは必要ない。

##### 1. Load the bim file, the summary statistic and the QC SNP list into R

In [14]:
cd ~/hmiwa1/storage/hmiwa-lab/03_231113_TutorialPRS/data
#1-4は連続して実行すること
cat <<EOL > ~/hmiwa1/storage/hmiwa-lab/03_231113_TutorialPRS/data/mismatchSNP.R
# Read in bim file
bim <- read.table("EUR.bim")
colnames(bim) <- c("CHR", "SNP", "CM", "BP", "B.A1", "B.A2")
# Read in QCed SNPs
qc <- read.table("EUR.QC.snplist", header = F, stringsAsFactors = F)
# Read in the GWAS data
height <-
    read.table(gzfile("Height.QC.gz"),
            header = T,
            stringsAsFactors = F, 
            sep="\t")
# Change all alleles to upper case for easy comparison
height\$A1 <- toupper(height\$A1)
height\$A2 <- toupper(height\$A2)
bim\$B.A1 <- toupper(bim\$B.A1)
bim\$B.A2 <- toupper(bim\$B.A2)
EOL

##### 2. Identify SNPs that require strand flipping

In [15]:
cat <<EOL >> ~/hmiwa1/storage/hmiwa-lab/03_231113_TutorialPRS/data/mismatchSNP.R
# Merge summary statistic with target
info <- merge(bim, height, by = c("SNP", "CHR", "BP"))
# Filter QCed SNPs
info <- info[info\$SNP %in% qc\$V1,]
# Function for finding the complementary allele
complement <- function(x) {
    switch (
        x,
        "A" = "T",
        "C" = "G",
        "T" = "A",
        "G" = "C",
        return(NA)
    )
}
# Get SNPs that have the same alleles across base and target
info.match <- subset(info, A1 == B.A1 & A2 == B.A2)
# Identify SNPs that are complementary between base and target
info\$C.A1 <- sapply(info\$B.A1, complement)
info\$C.A2 <- sapply(info\$B.A2, complement)
info.complement <- subset(info, A1 == C.A1 & A2 == C.A2)
# Update the complementary alleles in the bim file
# This allow us to match the allele in subsequent analysis
complement.snps <- bim\$SNP %in% info.complement\$SNP
bim[complement.snps,]\$B.A1 <-
    sapply(bim[complement.snps,]\$B.A1, complement)
bim[complement.snps,]\$B.A2 <-
    sapply(bim[complement.snps,]\$B.A2, complement)
EOL

##### 3. Identify SNPs that require recoding in the target (to ensure the coding allele in the target data is the effective allele in the base summary statistic)

In [16]:
cat <<EOL >> ~/hmiwa1/storage/hmiwa-lab/03_231113_TutorialPRS/data/mismatchSNP.R
# identify SNPs that need recoding
info.recode <- subset(info, A1 == B.A2 & A2 == B.A1)
# Update the recode SNPs
recode.snps <- bim\$SNP %in% info.recode\$SNP
tmp <- bim[recode.snps,]\$B.A1
bim[recode.snps,]\$B.A1 <- bim[recode.snps,]\$B.A2
bim[recode.snps,]\$B.A2 <- tmp

# identify SNPs that need recoding & complement
info.crecode <- subset(info, A1 == C.A2 & A2 == C.A1)
# Update the recode + strand flip SNPs
com.snps <- bim\$SNP %in% info.crecode\$SNP
tmp <- bim[com.snps,]\$B.A1
bim[com.snps,]\$B.A1 <- as.character(sapply(bim[com.snps,]\$B.A2, complement))
bim[com.snps,]\$B.A2 <- as.character(sapply(tmp, complement))

# Output updated bim file
write.table(
    bim[,c("SNP", "B.A1")],
    "EUR.a1",
    quote = F,
    row.names = F,
    col.names = F,
    sep="\t"
)
EOL

##### 4. Identify SNPs that have different allele in base and target (usually due to difference in genome build or Indel)

In [17]:
cat <<EOL >> ~/hmiwa1/storage/hmiwa-lab/03_231113_TutorialPRS/data/mismatchSNP.R
mismatch <-
    bim\$SNP[!(bim\$SNP %in% info.match\$SNP |
                bim\$SNP %in% info.complement\$SNP | 
                bim\$SNP %in% info.recode\$SNP |
                bim\$SNP %in% info.crecode\$SNP)]
write.table(
    mismatch,
    "EUR.mismatch",
    quote = F,
    row.names = F,
    col.names = F
)
EOL

In [18]:
cat ~/hmiwa1/storage/hmiwa-lab/03_231113_TutorialPRS/data/mismatchSNP.R

# Read in bim file
bim <- read.table("EUR.bim")
colnames(bim) <- c("CHR", "SNP", "CM", "BP", "B.A1", "B.A2")
# Read in QCed SNPs
qc <- read.table("EUR.QC.snplist", header = F, stringsAsFactors = F)
# Read in the GWAS data
height <-
    read.table(gzfile("Height.QC.gz"),
            header = T,
            stringsAsFactors = F, 
            sep="\t")
# Change all alleles to upper case for easy comparison
height$A1 <- toupper(height$A1)
height$A2 <- toupper(height$A2)
bim$B.A1 <- toupper(bim$B.A1)
bim$B.A2 <- toupper(bim$B.A2)
# Merge summary statistic with target
info <- merge(bim, height, by = c("SNP", "CHR", "BP"))
# Filter QCed SNPs
info <- info[info$SNP %in% qc$V1,]
# Function for finding the complementary allele
complement <- function(x) {
    switch (
        x,
        "A" = "T",
        "C" = "G",
        "T" = "A",
        "G" = "C",
        return(NA)
    )
}
# Get SNPs that have the same alleles across base and target
info.match <- subset(info, A1 == B.A1 & A2 == B.A2)
# Iden

In [19]:
Rscript ~/hmiwa1/storage/hmiwa-lab/03_231113_TutorialPRS/data/mismatchSNP.R

In [20]:
ls ~/hmiwa1/storage/hmiwa-lab/03_231113_TutorialPRS/data/

EUR.a1   EUR.height    EUR.QC.irem       EUR.valid.sample    Height.QC.gz
EUR.bed  EUR.mismatch  EUR.QC.log        EUR.zip             het.R
EUR.bim  EUR.QC.fam    EUR.QC.prune.in   Height.gwas.txt.gz  mismatchSNP.R
EUR.cov  EUR.QC.het    EUR.QC.prune.out  Height.gz
EUR.fam  EUR.QC.hh     EUR.QC.snplist    Height.nodup.gz


#### Duplicate SNPs
Make sure to remove any duplicate SNPs in your target data (these target data were simulated and so include no duplicated SNPs).

ターゲットデータ中の重複SNPを必ず削除してください（これらのターゲットデータはシミュレートされたため、重複SNPは含まれていません）。
#### Sex chromosomes
Sometimes sample mislabelling can occur, which may lead to invalid results. One indication of a mislabelled sample is a difference between reported sex and that indicated by the sex chromosomes. While this may be due to a difference in sex and gender identity, it could also reflect mislabeling of samples or misreporting and, thus, individuals in which there is a mismatch between biological and reported sex are typically removed. A sex check can be performed in PLINK, in which individuals are called as females if their X chromosome homozygosity estimate (F statistic) is < 0.2 and as males if the estimate is > 0.8.

時には検体の誤ラベリングが起こり、無効な結果につながることがあります。誤ラベリングされたサンプルの兆候の一つは、報告された性と性染色体が示す性との違いである。これは性別や性自認の違いに起因する場合もありますが、サンプルの誤ラベリングや誤った報告を反映している可能性もあり、そのため、生物学的性別と報告された性別の間に不一致がある個体は通常除外されます。性チェックはPLINKで行うことができ、X染色体のホモ接合性の推定値（F統計量）が0.2未満の個体はメス、0.8以上の個体はオスと呼ばれる。

Before performing a sex check, pruning should be performed (see here). A sex check can then easily be conducted using plink

性チェックを行う前に、剪定を行うべきである（こちらを参照）。その後、plinkを使って簡単に雌雄チェックを行うことができる。

In [21]:
plink \
    --bfile EUR \
    --extract EUR.QC.prune.in \
    --keep EUR.valid.sample \
    --check-sex \
    --out EUR.QC

PLINK v1.90b7 64-bit (16 Jan 2023)             www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to EUR.QC.log.
Options in effect:
  --bfile EUR
  --check-sex
  --extract EUR.QC.prune.in
  --keep EUR.valid.sample
  --out EUR.QC

257564 MB RAM detected; reserving 128782 MB for main workspace.
551892 variants loaded from .bim file.
503 people (240 males, 263 females) loaded from .fam.
--extract: 268457 variants remaining.
--keep: 487 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 487 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
treat these as missing.
Total genotyping rate in remaining samples is 0.999958.
268457 variants and 487 people pass filters and QC.


This will generate a file called EUR.QC.sexcheck containing the F-statistics for each individual. Individuals are typically called as being biologically male if the F-statistic is > 0.8 and biologically female if F < 0.2.

これにより、各個体のF統計量を含むEUR.QC.sexcheckというファイルが生成される。F統計量が0.8以上の個体は生物学的に男性であり、0.2未満の個体は生物学的に女性であると通常呼ばれます。

In [22]:
cat <<EOL >> ~/hmiwa1/storage/hmiwa-lab/03_231113_TutorialPRS/data/sexcheck.R
# Read in file
valid <- read.table("EUR.valid.sample", header=T)
dat <- read.table("EUR.QC.sexcheck", header=T)
valid <- subset(dat, STATUS=="OK" & FID %in% valid\$FID)
write.table(valid[,c("FID", "IID")], "EUR.QC.valid", row.names=F, col.names=F, sep="\t", quote=F) 
EOL

In [23]:
cat ~/hmiwa1/storage/hmiwa-lab/03_231113_TutorialPRS/data/sexcheck.R

# Read in file
valid <- read.table("EUR.valid.sample", header=T)
dat <- read.table("EUR.QC.sexcheck", header=T)
valid <- subset(dat, STATUS=="OK" & FID %in% valid$FID)
write.table(valid[,c("FID", "IID")], "EUR.QC.valid", row.names=F, col.names=F, sep="\t", quote=F) 


In [24]:
Rscript ~/hmiwa1/storage/hmiwa-lab/03_231113_TutorialPRS/data/sexcheck.R

In [25]:
ls

EUR.a1      EUR.mismatch  EUR.QC.prune.in   EUR.zip             mismatchSNP.R
EUR.bed     EUR.QC.fam    EUR.QC.prune.out  Height.gwas.txt.gz  sexcheck.R
EUR.bim     EUR.QC.het    EUR.QC.sexcheck   Height.gz
EUR.cov     EUR.QC.hh     EUR.QC.snplist    Height.nodup.gz
EUR.fam     EUR.QC.irem   EUR.QC.valid      Height.QC.gz
EUR.height  EUR.QC.log    EUR.valid.sample  het.R


In [29]:
head EUR.QC.valid

HG00096	HG00096
HG00097	HG00097
HG00099	HG00099
HG00101	HG00101
HG00102	HG00102
HG00103	HG00103
HG00105	HG00105
HG00107	HG00107
HG00108	HG00108
HG00109	HG00109


In [30]:
head EUR.QC.sexcheck

      FID       IID       PEDSEX       SNPSEX       STATUS            F
  HG00096   HG00096            1            1           OK            1
  HG00097   HG00097            2            2           OK       -0.217
  HG00099   HG00099            2            2           OK      -0.1848
  HG00101   HG00101            1            1           OK            1
  HG00102   HG00102            2            2           OK      -0.3558
  HG00103   HG00103            1            1           OK            1
  HG00105   HG00105            1            1           OK            1
  HG00107   HG00107            1            1           OK            1
  HG00108   HG00108            1            1           OK            1


In [31]:
wc EUR.QC.valid
wc EUR.QC.sexcheck

 483  966 7728 EUR.QC.valid
  488  2928 35136 EUR.QC.sexcheck


#### Sample overlap
Since the target data were simulated there are no overlapping samples between the base and target data here (see the relevant section of the paper for discussion of the importance of avoiding sample overlap).

ターゲットデータはシミュレートされたものであるため、ベースデータとターゲットデータの間に重複するサンプルはない（サンプルの重複を避けることの重要性については、論文の関連セクションを参照）。
#### Relatedness
Closely related individuals in the target data may lead to overfitted results, limiting the generalisability of the results.
対象データ中の近縁の個体は、過剰適合を引き起こし、結果の一般性を制限する可能性がある。


Before calculating the relatedness, pruning should be performed (see here). Individuals that have a first or second degree relative in the sample
π
>0.125
) can be removed with the following co
近縁度を計算する前に、プルーニングを行うべきである（こちらを参照）。サンプル中に1度または2度の近親者がいる個体（π>0.125）は、以下のコマンドで削除できます：ta file

In [32]:
plink \
    --bfile EUR \
    --extract EUR.QC.prune.in \
    --keep EUR.QC.valid \
    --rel-cutoff 0.125 \
    --out EUR.QC

PLINK v1.90b7 64-bit (16 Jan 2023)             www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to EUR.QC.log.
Options in effect:
  --bfile EUR
  --extract EUR.QC.prune.in
  --keep EUR.QC.valid
  --out EUR.QC
  --rel-cutoff 0.125

257564 MB RAM detected; reserving 128782 MB for main workspace.
551892 variants loaded from .bim file.
503 people (240 males, 263 females) loaded from .fam.
--extract: 268457 variants remaining.
--keep: 483 people remaining.
Using up to 63 threads (change this with --threads).
Before main variant filters, 483 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.999957.
268457 variants and 483 people pass filters and QC (before --rel-cutoff).
No

In [33]:
ls

EUR.a1      EUR.mismatch  EUR.QC.prune.in   EUR.valid.sample    het.R
EUR.bed     EUR.QC.fam    EUR.QC.prune.out  EUR.zip             mismatchSNP.R
EUR.bim     EUR.QC.het    EUR.QC.rel.id     Height.gwas.txt.gz  sexcheck.R
EUR.cov     EUR.QC.hh     EUR.QC.sexcheck   Height.gz
EUR.fam     EUR.QC.irem   EUR.QC.snplist    Height.nodup.gz
EUR.height  EUR.QC.log    EUR.QC.valid      Height.QC.gz


In [34]:
head EUR.QC.rel.id
wc EUR.QC.rel.id

HG00096	HG00096
HG00097	HG00097
HG00099	HG00099
HG00101	HG00101
HG00102	HG00102
HG00103	HG00103
HG00105	HG00105
HG00107	HG00107
HG00108	HG00108
HG00109	HG00109
 483  966 7728 EUR.QC.rel.id


>A greedy algorithm is used to remove closely related individuals in a way that optimizes the size of the sample retained. However, the algorithm is dependent on the random seed used, which can generate different results. Therefore, to reproduce the same result, you will need to specify the same random seed.
>>貪欲なアルゴリズムは、保持されるサンプルのサイズを最適化する方法で、近縁の個体を除去するために使用される。しかし、このアルゴリズムは使用するランダムシードに依存し、異なる結果を生成することがあります。したがって、同じ結果を再現するには、同じランダム・シードを指定する必要があります。

>PLINK's algorithm for removing related individuals does not account for the phenotype under study. To minimize the removal of cases of a disease, the following algorithm can be used instead: GreedyRelated.
>
>近縁個体を除去するPLINKのアルゴリズムは、調査対象の表現型を考慮していません。病気のケースの除去を最小にするために、代わりに以下のアルゴリズムを使用できます： GreedyRelated.le

### Generate final QC'ed target data file
After performing the full analysis, you can generate a QC'ed data set with the following command:

完全な分析を行った後、以下のコマンドでQCされたデータセットを生成することができる：

In [35]:
plink \
    --bfile EUR \
    --make-bed \
    --keep EUR.QC.rel.id \
    --out EUR.QC \
    --extract EUR.QC.snplist \
    --exclude EUR.mismatch \
    --a1-allele EUR.a1

PLINK v1.90b7 64-bit (16 Jan 2023)             www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to EUR.QC.log.
Options in effect:
  --a1-allele EUR.a1
  --bfile EUR
  --exclude EUR.mismatch
  --extract EUR.QC.snplist
  --keep EUR.QC.rel.id
  --make-bed
  --out EUR.QC

257564 MB RAM detected; reserving 128782 MB for main workspace.
551892 variants loaded from .bim file.
503 people (240 males, 263 females) loaded from .fam.
--extract: 540534 variants remaining.
--exclude: 489805 variants remaining.
--keep: 483 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 483 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is exactly

In [36]:
ls

EUR.a1      EUR.mismatch  EUR.QC.irem       EUR.QC.snplist      Height.nodup.gz
EUR.bed     EUR.QC.bed    EUR.QC.log        EUR.QC.valid        Height.QC.gz
EUR.bim     EUR.QC.bim    EUR.QC.prune.in   EUR.valid.sample    het.R
EUR.cov     EUR.QC.fam    EUR.QC.prune.out  EUR.zip             mismatchSNP.R
EUR.fam     EUR.QC.het    EUR.QC.rel.id     Height.gwas.txt.gz  sexcheck.R
EUR.height  EUR.QC.hh     EUR.QC.sexcheck   Height.gz
